In [11]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output

from nn import Net
from utils import *
from layer import Layer
from optimisers import *
from activations import *

import keras
from keras.datasets import mnist
from keras.datasets import cifar10

In [25]:
num_classes = 2
classes = (3,7) # np.random.choice(np.arange(10), num_classes, replace=False)

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(len(x_train), -1)/255
x_test  =  x_test.reshape(len(x_test),  -1)/255

inds_train = [y in classes for y in y_train]
inds_test  = [y in classes for y in y_test]

y_train, y_test = y_train.flatten(), y_test.flatten()
x_train, x_test = x_train[inds_train], x_test[inds_test]

y_test  = to_categorical(y_test[inds_test]).astype(np.int)
y_train = to_categorical(y_train[inds_train]).astype(np.int)

In [37]:
num_classes = 2
classes = (0,1)
x_train = np.random.uniform(-1, 1, (1000,100))
y_train = np.array([[0,1], [1,0]]*500)

x_test = np.random.uniform(-1, 1, (1000,100))
y_test = np.array([[0,1], [1,0]]*500)

print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(1000, 100) (1000, 2) (1000, 100) (1000, 2)


In [47]:
net = Net()
net.add_layer(Layer(act=tanh, output_size=50, input_size=100))
net.add_layer(Layer(act=tanh, output_size=50))
net.add_layer(Layer(act=tanh, output_size=50))
net.add_layer(Layer(act=softmax, output_size=num_classes))

net.compile(optimiser=Adam())

num_epochs = 3000
batch_size = 999

neurons, lyrs = 5, 3

shuffled_inds = np.arange(len(x_train))
log = LogPrint(["train_acc", "test_acc", "train_loss","test_loss"])

for epoch in range(num_epochs):
    test_acc, test_loss, train_ginni = net.metrics(x_test, y_test)
    train_acc, train_loss, test_ginni = net.metrics(x_train, y_train)
#     log.print((train_acc,test_acc,train_loss,test_loss))
    for i in range(0, len(x_train)-batch_size, batch_size):
        batch_inds = np.arange(i,i+batch_size-1)
        x_batch, y_batch = x_train[batch_inds], y_train[batch_inds]

        net.forward_pass(x_batch)
        net.backward_pass(y_batch)
        
        if i % 1 == 0:
            class_inds = [np.where(y_batch[:,i])[0] for i in range(y_batch.shape[-1])]
            fig, axarr = plt.subplots(nrows=lyrs, ncols=neurons, sharex=True, sharey=True,figsize=(15, 8))

            clear_output(wait=True)
            for lyr in range(lyrs):
                for neuron in range(neurons):
                    for inds in class_inds:
                        axarr[lyr, neuron].set_xlim((-1.1,1.1))
                        axarr[lyr, neuron].set_ylim((0,batch_size//2))
                        h=axarr[lyr, neuron].hist(net.layers[lyr].activations[:,neuron][inds], bins=25,alpha=0.6)
#                         axarr[lyr, neuron].plot(h[1][1:],h[0])
            fig.suptitle("Epoch %d, iteration %d \nTrain acc - %f \nTest acc - %f"%(epoch,i,train_acc,test_acc))
            plt.savefig("./pics/_img_%d_%s"%(epoch, format(i, '015b')), transparent=False)
            plt.show()

    np.random.shuffle(shuffled_inds)

KeyboardInterrupt: 